In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

W0720 10:34:29.588000 7768 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


In [2]:
encoder = SentenceTransformer("all-MiniLM-L6-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\amlan\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\amlan\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
import pandas as pd

desc_df = pd.read_csv('../description.csv')
desc_df.head()

,img,description
0,00000_00.jpg,The dress is a white Levi's T-shirt with a red...
1,00001_00.jpg,The dress is a black t-shirt with a Levi's log...
2,00002_00.jpg,The dress is a black long-sleeved top with a r...
3,00003_00.jpg,The dress is a navy blue t-shirt with a pocket...
4,00005_00.jpg,The dress is a white Levi's T-shirt with a red...


In [7]:
to_emb = desc_df['description'].tolist()

In [8]:
to_emb

["The dress is a white Levi's T-shirt with a red logo on the front and blue jeans with a button and a small tear at the waist.",
 "The dress is a black t-shirt with a Levi's logo on the left chest and black leather pants with a belt.",
 'The dress is a black long-sleeved top with a round neckline and fitted fit, paired with blue jeans.',
 'The dress is a navy blue t-shirt with a pocket and black jeans.',
 "The dress is a white Levi's T-shirt with a red logo on the front and blue and white striped pants.",
 'The dress is a pastel purple Adidas Originals T-shirt with white stripes on the sleeves and collar, paired with matching Adidas Originals pants.',
 'The dress is a black cropped t-shirt with white stripes on the sleeves and an orange mini skirt with a front zipper and side pockets.',
 'The dress is a Calvin Klein Jeans black T-shirt paired with beige pants.',
 'The dress is a black Adidas t-shirt with a sheer skirt featuring white stripes on the sleeves and waistband.',
 'The dress 

In [9]:
embeddings = encoder.encode(to_emb, show_progress_bar=True)

Batches:   0%|          | 0/135 [00:00<?, ?it/s]

In [11]:
type(embeddings)

numpy.ndarray

In [12]:
import torch

emb_tensor: torch.Tensor = torch.from_numpy(embeddings)

In [15]:
type(emb_tensor)

torch.Tensor

In [16]:
desc_df['emb'] = list(emb_tensor)

In [17]:
desc_df.head()

,img,description,emb
0,00000_00.jpg,The dress is a white Levi's T-shirt with a red...,"[tensor(-0.0309), tensor(0.0942), tensor(0.038..."
1,00001_00.jpg,The dress is a black t-shirt with a Levi's log...,"[tensor(-0.0310), tensor(0.1181), tensor(-0.03..."
2,00002_00.jpg,The dress is a black long-sleeved top with a r...,"[tensor(-0.0430), tensor(0.0838), tensor(-0.00..."
3,00003_00.jpg,The dress is a navy blue t-shirt with a pocket...,"[tensor(-0.0382), tensor(0.0527), tensor(0.004..."
4,00005_00.jpg,The dress is a white Levi's T-shirt with a red...,"[tensor(-0.0436), tensor(0.0918), tensor(-0.01..."


In [23]:
embedding_length_desc = len(emb_tensor[0])
embedding_length_desc

384

In [24]:
emb_tensor.shape

torch.Size([4313, 384])

In [25]:
import numpy as np

np.save('desc_embeddings.npy', embeddings)

In [26]:
cloth_emb = torch.from_numpy(np.load('../embeddings/cloth_emb_swin.npy'))
image_emb = torch.from_numpy(np.load('../embeddings/image_emb_swin.npy'))

In [27]:
fused_emb = torch.cat([image_emb, cloth_emb], dim=-1)

In [28]:
embedding_length_img = len(fused_emb[0])
embedding_length_img

2048